In [1]:
from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import seaborn as sns

import shared_utils
import os

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import rt_filter_map_plot

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# May 4

In [7]:
working_organizations = pd.read_parquet('./2022-05-04_working_organizations.parquet')

In [10]:
working_organizations

,itp_id,name,caltrans_district,website,ntp_id,drmt_organization_name,analysis_date
676,135,Humboldt Transit Authority,01 - Eureka,https://hta.org/,['reccKueWkvBCZoLQs'],['recu9h9nAElLpYoYO'],2022-05-04
715,159,Lake Transit Authority,01 - Eureka,https://laketransit.org/,['recF9AnmCtEyxxDTY'],['reck7ddrpXSeA0mH0'],2022-05-04
584,259,Redding Area Bus Authority,02 - Redding,https://www.cityofredding.org/departments/redd...,['recoXZjKEOPldH8QJ'],None,2022-05-04
653,221,Nevada County,03 - Marysville,https://www.mynevadacounty.com/,['recx7U6zAWSOGpcyR'],None,2022-05-04
669,372,Yolo County Transportation District,03 - Marysville,http://www.yolobus.com/,['recYR6UZWz55HVYzD'],None,2022-05-04
535,194,Marin County Transit District,04 - Oakland,https://marintransit.org/,['recgeJopomk1yuLGl'],None,2022-05-04
552,315,Sonoma-Marin Area Rail Transit District,04 - Oakland,https://www.sonomamarintrain.org/,['recguX2t9zxBKD7RR'],None,2022-05-04
574,246,Peninsula Corridor Joint Powers Board,04 - Oakland,https://www.caltrain.com/,['rec1cXuPDvqFKBYfN'],None,2022-05-04
623,110,City of Fairfield,04 - Oakland,https://fasttransit.org/,['recYcSIprn4TUbpTq'],None,2022-05-04
628,247,City of Petaluma,04 - Oakland,https://transit.cityofpetaluma.net/,['recguWshX2KB6uhKC'],None,2022-05-04


In [8]:
def aggregate_endpoint_delays(working_organizations):
    endpoint_delay_all = pd.DataFrame()
    for _ix, operator in working_organizations.iterrows():
        itp_id = operator.itp_id
        analysis_date = operator.analysis_date
        rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)
        df = rt_day.endpoint_delay_view
        df['calitp_itp_id'] = itp_id
        endpoint_delay_all = pd.concat([endpoint_delay_all, df])
    return endpoint_delay_all

In [9]:
may_endpoint_delays = aggregate_endpoint_delays(working_organizations)

FileNotFoundError: calitp-analytics-data/data-analyses/rt_delay/rt_trips/135_05_04.parquet

In [ ]:
may_endpoint_delays >> head(3)

In [24]:
may_delay_hours = may_endpoint_delays.delay_seconds.sum() / 60 ** 2
may_delay_hours

5177.317222222222

In [25]:
may_delay_hours / may_endpoint_delays.shape[0]

0.0949792188996922

# Feb 8

In [13]:
working_organizations = pd.read_parquet('./2022-02-08_working_organizations.parquet')

In [15]:
feb_endpoint_delays = aggregate_endpoint_delays(working_organizations)

found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet
found parquet


In [16]:
feb_endpoint_delays >> head(3)

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id,actual_time,delay_seconds,arrival_hour,mean_speed_mph,calitp_itp_id
0,1264,Valley West Blvd (McDonald's),POINT (-344115.784 328111.976),p_530998,12463.119679,4231438015864381462,t_1229664_b_18361_tn_0,6,2022-02-08 16:45:00,12416,BLRTS,1,2022-02-08 16:41:01.997936,0,16,85.971007,135
1,1264,Valley West Blvd (McDonald's),POINT (-344115.784 328111.976),p_414,4768.171008,2215492193602828897,t_2062941_b_33847_tn_0,2,2022-02-08 08:30:00,8,None,1,2022-02-08 08:30:13.627436,13,8,39.181819,135
2,802640,Redcrest Off Ramp,POINT (-334985.395 271623.374),p_179043,57567.272370,-7983805054614466631,t_2062926_b_33847_tn_0,12,2022-02-08 12:13:00,822,None,1,2022-02-08 12:15:49.925327,169,12,38.096445,135


In [18]:
feb_delay_hours = feb_endpoint_delays.delay_seconds.sum() / 60 ** 2
feb_delay_hours

4647.175833333334

In [20]:
feb_delay_hours / feb_endpoint_delays.shape[0]

0.11649977020138716

# Old

In [21]:
endpoint_delay_all = pd.DataFrame()
for operator_parquet in os.listdir('./endpoint_delay_views/'):
    if operator_parquet == '.ipynb_checkpoints':
        continue
    calitp_itp_id = operator_parquet.split('_')[0]
    df = gpd.read_parquet(f'./endpoint_delay_views/{operator_parquet}')
    df['calitp_itp_id'] = int(calitp_itp_id)
    endpoint_delay_all = pd.concat([endpoint_delay_all, df])

In [22]:
endpoint_delay_all >> head(5)

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,direction_id,actual_time,arrival_hour,mean_speed_mph,delay_seconds,calitp_itp_id
0,13171,Genesee Av & La Jolla Village Dr,POINT (261001.239 -567362.657),3010093,45232.728222,-1879550975976699090,16174935-NC2110-NCTD-Weekday-20,73,2022-02-08 07:43:00,301,1,2022-02-08 07:44:07.859503,7,18.560864,67,226
1,13171,Genesee Av & La Jolla Village Dr,POINT (261001.239 -567362.657),3010093,45232.728222,-7086145044901280522,16174928-NC2110-NCTD-Weekday-20,73,2022-02-08 18:59:00,301,1,2022-02-08 18:58:22.016938,18,15.325906,0,226
2,13171,Genesee Av & La Jolla Village Dr,POINT (261001.239 -567362.657),3010093,45232.728222,-2987294244121688089,16174910-NC2110-NCTD-Weekday-20,73,2022-02-08 12:27:00,301,1,2022-02-08 12:29:09.865643,12,15.733527,129,226
3,13171,Genesee Av & La Jolla Village Dr,POINT (261001.239 -567362.657),3010093,45232.728222,-1093749300821038708,16174906-NC2110-NCTD-Weekday-20,73,2022-02-08 22:15:00,301,1,2022-02-08 22:16:39.039018,22,17.243248,99,226
4,13171,Genesee Av & La Jolla Village Dr,POINT (261001.239 -567362.657),3010093,45232.728222,8517315111569206284,16174896-NC2110-NCTD-Weekday-20,73,2022-02-08 19:51:00,301,1,2022-02-08 19:48:15.121243,19,17.204731,0,226


In [23]:
endpoint_delay_all.delay_seconds.sum() / 60 ** 2 ## ~3340 hours of delay on a weekday, statewide

3338.7069444444446

In [4]:
gdf = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/bus_service_increase/parallel_or_intersecting_2022-05-04.parquet')

In [5]:
parallel_routes = (gdf
                   >> filter(_.pct_route >= .3)
                   >> select(_.calitp_itp_id == _.itp_id, _.route_id)
                   >> distinct(_.calitp_itp_id, _.route_id)
                  )

In [26]:
parallel_delays = (endpoint_delay_all
                   >> inner_join(_, parallel_routes, on = ['calitp_itp_id', 'route_id'])
                   >> distinct(_.calitp_itp_id, _.trip_id, _keep_all=True)
                   ## ensure we don't duplicate delays...
                   ## guessing these come partially from multiple url numbers/aggregation in RT tools?
                  )

In [27]:
parallel_delays.delay_seconds.sum() / 60 ** 2

1745.3352777777777

In [28]:
parallel_delays.route_id.isin(gdf.route_id).all() ## looks good

True

#### what about parallel routes without RT data?

In [29]:
parallel_routes >> distinct(_.calitp_itp_id, _.route_id) >> filter(_.calitp_itp_id != 200)

,calitp_itp_id,route_id
0,4,10
1,4,12
2,4,14
3,4,18
4,4,19
...,...,...
3103,482,30
3104,482,31
3105,482,32
3106,482,7


In [14]:
all_ca_routes = tbl.gtfs_schedule.routes() >> distinct(_.calitp_itp_id, _.route_id) >> collect()

/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)


In [19]:
all_ca_routes >> filter(_.calitp_itp_id != 200)

,calitp_itp_id,route_id
0,279,12
1,279,2
2,279,11
3,279,4
4,279,6
...,...,...
3992,372,060f6b21-00e0-46a1-8be8-98ed29ed5673
3993,372,96926054-eff8-4292-975d-f9917fe88820
3994,372,232750c4-edd8-467e-8c8a-0b860672803e
3995,372,dcf17c80-34f0-45e1-861f-c53760ac2d3c


In [17]:
parallel_delays >> distinct(_.calitp_itp_id, _.route_id)

,calitp_itp_id,route_id
0,226,301
1,226,398
2,226,323
3,226,332
4,226,354
...,...,...
932,300,3366
933,300,3369
934,300,3354
935,300,3374
